# Lab 5

In [1]:
import MNIST 
import numpy as np
import sklearn.preprocessing
from numpy import linalg as LA
from future.utils import iteritems
from collections import defaultdict
from sklearn import metrics, datasets
from sklearn.naive_bayes import GaussianNB
from scipy.stats import multivariate_normal as mvn

In this lab session you will
+ explore some programming concepts in Python, SciKitLearn, and Numpy to implement several statistical / Bayesian classifiers and compare them against each other and with one provided in SciKitLearn,
+ get acquainted with another version of the MNIST dataset, and 
+ explore the effects of specific phenomena in the data on the classification results, also depending on the choice of discrete vs continuous implementation of the models.

### Classifiers

#### 1. Gaussian NB Classifier

Make use of the provided Gaussian NB Classifier (sklearn.naive_bayes GaussianNB) for all data sets as a comparison. It is already implemented in the handout for the MNIST_Light set (see below).

gnb = GaussianNB()

#### 2. Nearest Centroid Classifier (NCC)

Implement your own Nearest Centroid Classifier (NCC): The NCC fit method should simply compute the mean values over the attribute values of the examples for each class. Prediction is then done by finding the argmin over the distances from the class centroids for each sample. This classifier should be run on all three variants of data sets, see below.

In [2]:
class NCC:
    def __init__(self):
        self.mean = {}
        
    def fit(self, X_train, y_train):
        
        inidices_labels = defaultdict(list)
        for i in range(len(y_train)):
            # Collect indices of exemplars for the given class label (keys)
            inidices_labels[y_train[i]].append(i)
                    
        for index, label in enumerate(inidices_labels.keys()):
            # subselect an array of exemplars associated with
            # this label
            matrix_same_class = X_train[inidices_labels[label]]
            
            # Compute centroid for a set of training vectors X_train, axis=0 since we want columnwise
            centroid = matrix_same_class.mean(axis=0)

            self.mean[label] = centroid
        return None
            
    def predict(self, X_test):
        y_pred = []
        for i in X_test:
            temp_vec = []
            
            # Predicting the outcome
            for j in self.mean.values():
                norm = LA.norm(i-j)
                temp_vec.append(norm)
            y_pred.append(np.argmin(temp_vec))
            
        # Needs to be an array    
        y_pred = np.array(y_pred)
        
        return y_pred

In [3]:
ncc = NCC()

#### 3.  Naive Bayesian Classifier (NBC)

Implement a Naive Bayesian Classifier (NBC) based on discrete (statistical) values (i.e., counts of examples falling into the different classes and attribute value groups) both for the priors and for the conditional probabilities. Run this on the two SciKitLearn digits data sets. It should also work with the (non-normalised) MNIST_Light set, but it will probably take a (very long) while and not give anything interesting really...

In [6]:
class NBC:
    
    def __init__(self):
        self.conditional = dict()
        self.priors = dict() # P(class)
    
    def fit(self, X_train, y_train): # smoothing for fixing the variance , smoothing=1e-2
        
        labels = set(y_train)
        for c in labels:
            temp = {}
            current_x = X_train[y_train == c]
            
            # Transpose matrix to get all columns to rows
            trans_matrix = current_x.T
            pixels = np.unique(current_x)
            
            # Computes the probability of each pixels occurance in a specific image
            for col,row in enumerate(trans_matrix):
                unique, counts = np.unique(row, return_counts=True)
                temp[col] =  dict(zip(unique, counts/len(current_x)))
            self.conditional[c] = temp
               
            # Computes the mean of each class
            self.priors[c] = float(len(y_train[y_train == c])) / len(y_train)
    
        return None 
            
    def predict(self, X_test):
        predicted_classes = []
        # p(c)*Prod(p(x_i|c))
        for image_pixels in X_test:
            temp = []
            for c in self.priors.keys():
                prob = self.priors[c]
                for col,value in enumerate(image_pixels):
                    try:
                        prob *= self.conditional[c][col][value]
                    except:
                        prob = 0
                temp.append(prob)
                
            predicted_classes.append(np.argmax(temp))
        
        y_pred = np.asarray(predicted_classes)
  
        return y_pred

In [7]:
nbc = NBC()    

#### 4. Gaussian Naive Bayesian Classifier (GNB)

Implement your own Gaussian Naive Bayesian Classifier (GNB) (assume priors for the classes based on counts and Gaussian distributions for the conditional probabilities) and make sure it works for all three data sets. You will most likely encounter problems due to the edge pixels having value 0.0 in practically ALL images in ALL data sets. A workaround is to add an epsilon to the variance. Why is that still working?

In [6]:
class GNB:
    
    def __init__(self):
        self.gaussians = dict()
        self.priors = dict()
    
    def fit(self, X_train, y_train, smoothing=1e-2): # smoothing for fixing the variance
        
        labels = set(y_train)
        for c in labels:
            current_x = X_train[y_train == c]
            # Dictionary with class as key and each key containg its corresponding mean and variance
            self.gaussians[c] = {
                'mean': current_x.mean(axis=0),
                'var': current_x.var(axis=0) + smoothing,
            }
            # Computes the mean
            self.priors[c] = float(len(y_train[y_train == c])) / len(y_train)
        
        return None # self.gaussians, self.priors
            
    def predict(self, X_test):
        # Builds the P matrix of size (number images, number of classes)
        row, col = X_test.shape
        G = len(self.gaussians)
        P = np.zeros((row, G))
        
        for c, g in iteritems(self.gaussians):
            mean, var = g['mean'], g['var']
            
            # We can choose the digit class using c = argmax_c(logP(x|c)+logP(c))
            # mvn.logpdf() since we wants log of the probability density function
            P[:,c] = mvn.logpdf(X_test, mean=mean, cov=var) + np.log(self.priors[c])
              
        return np.argmax(P, axis=1)

In [7]:
GNB = GNB()

### Data sets

#### 1. SciKitLearn digits

SciKitLearn digits: just load it and use it as is. Use 70% of the data for training and 30% for testing. Run all of the four classifiers and compare and discuss the results.


In [8]:
digits = datasets.load_digits()
num_examples_digits = len(digits.data)
num_split_digits = int(0.7*num_examples_digits)
train_features_digits = digits.data[:num_split_digits]
train_labels_digits = digits.target[:num_split_digits]
test_features_digits = digits.data[num_split_digits:]
test_labels_digits = digits.target[num_split_digits:]

Gaussian NB Classifier

In [96]:
gnb.fit(train_features_digits, train_labels_digits)
y_pred_digits = gnb.predict(test_features_digits)

print("Classification report SKLearn GNB:\n%s\n"
  % (metrics.classification_report(test_labels_digits, y_pred_digits)))
print("Confusion matrix SKLearn GNB:\n%s" % metrics.confusion_matrix(test_labels_digits, y_pred_digits))

# mnist.visualize_wrong_class(y_pred_digits, 8)

Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        53
           1       0.65      0.79      0.71        53
           2       0.98      0.81      0.89        53
           3       0.92      0.64      0.76        53
           4       1.00      0.86      0.92        57
           5       0.83      0.93      0.87        56
           6       0.96      0.98      0.97        54
           7       0.73      0.83      0.78        54
           8       0.59      0.71      0.64        52
           9       0.82      0.73      0.77        55

    accuracy                           0.83       540
   macro avg       0.84      0.82      0.83       540
weighted avg       0.84      0.83      0.83       540


Confusion matrix SKLearn GNB:
[[51  0  0  0  0  0  0  0  2  0]
 [ 0 42  1  0  0  0  0  0  3  7]
 [ 0  5 43  1  0  0  1  0  1  2]
 [ 0  3  0 34  0  3  0  2 11  0]
 [ 1  0  0  0 49  0  0  6  1  0]
 [ 0  2  0

Nearest Centroid Classifier (NCC)

In [143]:
ncc.fit(train_features_digits, train_labels_digits)
y_pred_digits = ncc.predict(test_features_digits)

print("Classification report SKLearn GNB:\n%s\n"
  % (metrics.classification_report(test_labels_digits, y_pred_digits)))
print("Confusion matrix SKLearn GNB:\n%s" % metrics.confusion_matrix(test_labels_digits, y_pred_digits))


Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        53
           1       0.89      0.75      0.82        53
           2       0.98      0.85      0.91        53
           3       0.84      0.77      0.80        53
           4       0.98      0.93      0.95        57
           5       0.86      0.89      0.88        56
           6       0.98      0.96      0.97        54
           7       0.90      0.96      0.93        54
           8       0.80      0.75      0.77        52
           9       0.65      0.87      0.74        55

    accuracy                           0.87       540
   macro avg       0.88      0.87      0.88       540
weighted avg       0.88      0.87      0.88       540


Confusion matrix SKLearn GNB:
[[52  0  0  0  1  0  0  0  0  0]
 [ 0 40  0  0  0  1  0  0  0 12]
 [ 1  0 45  6  0  0  0  0  0  1]
 [ 0  1  0 41  0  2  0  3  5  1]
 [ 1  0  0  0 53  0  0  0  3  0]
 [ 0  0  0

Naive Bayesian Classifier (NBC)

In [9]:
nbc.fit(train_features_digits, train_labels_digits)
y_pred_digits = nbc.predict(test_features_digits)

print("Classification report SKLearn NBC:\n%s\n"
  % (metrics.classification_report(test_labels_digits, y_pred_digits)))
print("Confusion matrix SKLearn NBC:\n%s" % metrics.confusion_matrix(test_labels_digits, y_pred_digits))


Classification report SKLearn NBC:
              precision    recall  f1-score   support

           0       0.24      1.00      0.39        53
           1       0.84      0.51      0.64        53
           2       1.00      0.36      0.53        53
           3       1.00      0.58      0.74        53
           4       0.95      0.65      0.77        57
           5       0.88      0.52      0.65        56
           6       1.00      0.65      0.79        54
           7       0.86      0.59      0.70        54
           8       0.72      0.65      0.69        52
           9       0.75      0.65      0.70        55

    accuracy                           0.62       540
   macro avg       0.83      0.62      0.66       540
weighted avg       0.83      0.62      0.66       540


Confusion matrix SKLearn NBC:
[[53  0  0  0  0  0  0  0  0  0]
 [20 27  0  0  1  0  0  0  3  2]
 [28  0 19  0  0  0  0  0  5  1]
 [13  0  0 31  0  3  0  1  3  2]
 [20  0  0  0 37  0  0  0  0  0]
 [19  2  0

Gaussian Naive Bayesian Classifier (GNB) 

In [35]:
GNB.fit(train_features_digits, train_labels_digits)
y_pred_digits = GNB.predict(test_features_digits)

print("Classification report SKLearn GNB:\n%s\n"
  % (metrics.classification_report(test_labels_digits, y_pred_digits)))
print("Confusion matrix SKLearn GNB:\n%s" % metrics.confusion_matrix(test_labels_digits, y_pred_digits))


X_test (540, 64)
(540,)
(540,)
(540,)
(540,)
(540,)
(540,)
(540,)
(540,)
(540,)
(540,)
P [4 0 5 3 6 9 6 2 7 5 4 4 7 2 8 2 3 5 7 9 5 4 8 8 4 9 0 8 9 8 0 9 2 3 4 5 6
 8 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 9 5 5 6 5 0 9 8 9 8 4 1 7
 7 3 5 1 0 0 2 1 7 8 2 0 1 2 6 3 3 7 3 3 4 6 6 6 4 9 1 5 0 9 6 2 8 9 0 0 1
 7 6 3 2 1 7 4 6 3 1 3 9 1 7 6 8 4 3 1 4 0 8 3 6 9 6 1 7 5 4 4 7 2 8 2 2 5
 7 9 5 4 8 8 4 9 0 8 0 9 2 3 4 5 6 7 9 9 0 9 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6
 7 8 9 0 9 5 5 6 5 0 9 8 3 8 4 9 7 7 3 5 9 0 0 2 2 7 8 2 0 9 2 6 3 3 7 3 3
 4 6 6 6 4 9 9 5 0 5 5 2 9 2 0 0 9 7 6 3 2 9 7 4 6 3 1 3 9 9 7 6 8 4 3 9 4
 0 5 3 6 9 6 9 7 5 4 4 7 2 9 2 2 5 7 9 5 4 8 8 4 9 0 8 9 8 0 1 2 3 4 5 9 2
 1 9 0 8 2 3 4 5 6 9 0 1 2 3 4 5 1 7 5 7 4 9 5 5 6 5 0 7 7 5 8 4 1 7 8 8 5
 1 4 0 1 2 8 8 1 0 1 2 6 8 8 8 7 8 7 6 6 6 7 9 1 5 4 9 1 2 8 0 1 7 6 3 2 1
 5 7 6 3 1 3 7 1 8 6 8 4 3 1 4 0 5 3 6 9 6 1 7 5 4 4 7 2 2 5 5 3 5 8 4 5 0
 8 7 7 0 8 2 8 4 5 6 7 8 9 0 1 2 8 4 5 6 7 8 9 0 1 2 5 4 5 6 7 8 9 0 9 5 5
 6 5 0 9 8 

#### 2. SciKitLearn digits summarised

SciKitLearn digits summarised: reduce the data set to only contain three values for the attributes, e.g., 0 for 'dark', 1 for 'grey' and 2 for 'light', with dark, grey and light corresponding to the values suggested in lab 2 (decision trees). Split again into 70% training and 30% test data.
Run all four classifiers on this set and compare the results. Why are they so different from those for the original data in particular for the NBC? Why do they decrease in accuracy for the GNB?

In [10]:
def comp(vec):
    """Sets the pixel values to either 0, 1 or 2, that is, dark, grey or light"""
    vec_new = np.empty([len(vec)]) 
    if len(vec) == 1: vec = [vec]
    for i,x in enumerate(vec):
        if x<=5: vec_new[i] = 0
        elif x<=10: vec_new[i] = 1
        else: vec_new[i] = 2
    return vec_new

In [11]:
train_features_comp = np.asarray([comp(i) for i in train_features_digits])
train_labels_comp = np.asarray(train_labels_digits)
test_features_comp = np.asarray([comp(i) for i in test_features_digits])
test_labels_comp = np.asarray((test_labels_digits))

Gaussian NB Classifier

In [158]:
gnb.fit(train_features_comp, train_labels_comp)
y_pred_comp = gnb.predict(test_features_comp)

print("Classification report SKLearn digits summarize GNB:\n%s\n"
  % (metrics.classification_report(test_labels_comp, y_pred_comp)))
print("Confusion matrix SKLearn digits summarize GNB:\n%s" % metrics.confusion_matrix(test_labels_comp, y_pred_comp))


Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.92      0.89      0.90        53
           1       0.93      0.26      0.41        53
           2       1.00      0.74      0.85        53
           3       0.97      0.66      0.79        53
           4       0.94      0.84      0.89        57
           5       0.86      0.57      0.69        56
           6       0.93      0.98      0.95        54
           7       0.64      0.89      0.74        54
           8       0.41      0.88      0.56        52
           9       0.63      0.76      0.69        55

    accuracy                           0.75       540
   macro avg       0.82      0.75      0.75       540
weighted avg       0.83      0.75      0.75       540


Confusion matrix SKLearn GNB:
[[47  0  0  0  2  1  0  0  3  0]
 [ 0 14  0  0  1  1  0  6 19 12]
 [ 0  0 39  0  0  0  0  0 12  2]
 [ 0  0  0 35  0  0  0  4 12  2]
 [ 1  0  0  0 48  0  3  4  1  0]
 [ 1  0  0

IndexError: index 540 is out of bounds for axis 0 with size 540

Nearest Centroid Classifier (NCC)

In [153]:
ncc.fit(train_features_comp, train_labels_comp)
y_pred_comp = ncc.predict(test_features_comp)

print("Classification report SKLearn digits summarize NCC:\n%s\n"
  % (metrics.classification_report(test_labels_comp, y_pred_comp)))
print("Confusion matrix SKLearn digits summarize NCC:\n%s" % metrics.confusion_matrix(test_labels_comp, y_pred_comp))


Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        53
           1       0.85      0.64      0.73        53
           2       0.93      0.81      0.87        53
           3       0.77      0.77      0.77        53
           4       0.96      0.93      0.95        57
           5       0.83      0.88      0.85        56
           6       0.96      0.96      0.96        54
           7       0.91      0.93      0.92        54
           8       0.77      0.83      0.80        52
           9       0.67      0.84      0.74        55

    accuracy                           0.86       540
   macro avg       0.86      0.85      0.86       540
weighted avg       0.86      0.86      0.86       540


Confusion matrix SKLearn GNB:
[[51  0  0  0  1  1  0  0  0  0]
 [ 0 34  0  1  0  1  0  0  4 13]
 [ 1  1 43  7  0  0  0  0  0  1]
 [ 0  1  1 41  0  2  0  3  4  1]
 [ 1  0  0  0 53  0  0  0  3  0]
 [ 0  1  0

Naive Bayesian Classifier (NBC)

In [12]:
nbc.fit(train_features_comp, train_labels_comp)
y_pred_comp = nbc.predict(test_features_comp)

print("Classification report SKLearn digits summarize NBC:\n%s\n"
  % (metrics.classification_report(test_labels_comp, y_pred_comp)))
print("Confusion matrix SKLearn digits summarize NBC:\n%s" % metrics.confusion_matrix(test_labels_comp, y_pred_comp))


Classification report SKLearn digits summarize NBC:
              precision    recall  f1-score   support

           0       0.84      0.79      0.82        53
           1       0.85      0.74      0.79        53
           2       0.96      0.89      0.92        53
           3       0.83      0.74      0.78        53
           4       0.95      0.95      0.95        57
           5       0.79      0.88      0.83        56
           6       0.98      0.87      0.92        54
           7       0.83      0.83      0.83        54
           8       0.72      0.83      0.77        52
           9       0.67      0.82      0.74        55

    accuracy                           0.83       540
   macro avg       0.84      0.83      0.83       540
weighted avg       0.84      0.83      0.84       540


Confusion matrix SKLearn digits summarize NBC:
[[42  0  0  1  3  4  0  1  2  0]
 [ 0 39  0  0  0  1  0  0  1 12]
 [ 2  0 47  3  0  1  0  0  0  0]
 [ 0  1  1 39  0  1  0  3  6  2]
 [ 0  0  

Gaussian Naive Bayesian Classifier (GNB) 

In [262]:
GNB.fit(train_features_comp, train_labels_comp)
y_pred_comp = GNB.predict(test_features_comp)

print("Classification report SKLearn digits summarize GNB:\n%s\n"
  % (metrics.classification_report(test_labels_comp, y_pred_comp)))
print("Confusion matrix SKLearn digits summarize GNB:\n%s" % metrics.confusion_matrix(test_labels_comp, y_pred_comp))


Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.96      0.91      0.93        53
           1       0.92      0.66      0.77        53
           2       0.96      0.96      0.96        53
           3       0.85      0.74      0.79        53
           4       0.92      0.95      0.93        57
           5       0.89      0.88      0.88        56
           6       0.98      0.96      0.97        54
           7       0.82      0.87      0.85        54
           8       0.66      0.87      0.75        52
           9       0.69      0.76      0.72        55

    accuracy                           0.86       540
   macro avg       0.87      0.85      0.86       540
weighted avg       0.87      0.86      0.86       540


Confusion matrix SKLearn GNB:
[[48  0  0  0  4  0  0  0  1  0]
 [ 0 35  0  0  1  1  0  0  4 12]
 [ 1  0 51  1  0  0  0  0  0  0]
 [ 0  0  1 39  0  1  0  3  8  1]
 [ 0  0  0  0 54  0  0  2  1  0]
 [ 0  1  0

#### 3. MNIST_Light

MNIST_Light: load the MINST_Light data set (see above), inspect the contents (data and specifically the target) carefully and split the set into 70% training and 30% test data if you do not use the code in the handout (based on MNIST.py) as is. If you use the provided code to retrieve the data set, the three-dimensional (20x20-pixels per image) arrays will have been reshaped to be of the same two dimensions as given in the digits set (one flattened array per image) and the values for the attributes (pixels) will have been normalised from [0 ... 255] to [0.0 ... 1.0]. If you do not normalise them, you can observe a problem with the GNB. What is this problem and why is it solved by normalisation of the data?

In [217]:
mnist = MNIST.MNISTData('MNIST_Light/*/*.png')
train_features_mnist, test_features_mnist, train_labels_mnist, test_labels_mnist = mnist.get_data()
# mnist.visualize_random()

Gaussian NB Classifier

In [194]:
gnb.fit(train_features_mnist, train_labels_mnist)
y_pred_mnist = gnb.predict(test_features_mnist)

print("Classification report MNIST_Light GNB:\n%s\n"
  % (metrics.classification_report(test_labels_mnist, y_pred_mnist)))
print("Confusion matrix MNIST_Light GNB:\n%s" % metrics.confusion_matrix(test_labels_mnist, y_pred_mnist))

Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.54      0.94      0.69       164
           1       0.71      0.94      0.81       152
           2       0.83      0.50      0.62       155
           3       0.83      0.53      0.65       154
           4       0.75      0.31      0.44       143
           5       0.67      0.16      0.25       141
           6       0.81      0.85      0.83       143
           7       0.83      0.82      0.83       158
           8       0.41      0.64      0.50       132
           9       0.60      0.84      0.70       158

    accuracy                           0.66      1500
   macro avg       0.70      0.65      0.63      1500
weighted avg       0.70      0.66      0.64      1500


Confusion matrix SKLearn GNB:
[[154   0   6   0   1   1   0   0   1   1]
 [  1 143   1   0   0   1   0   1   3   2]
 [ 11   6  77  10   2   1  19   1  27   1]
 [ 32  11   5  82   0   0   2   3  12   7]
 [ 

Nearest Centroid Classifier (NCC)

In [156]:
ncc.fit(train_features_mnist, train_labels_mnist)
y_pred_mnist = ncc.predict(test_features_mnist)

print("Classification report MNIST_Light NCC:\n%s\n"
  % (metrics.classification_report(test_labels_mnist, y_pred_mnist)))
print("Confusion matrix MNIST_Light NCC:\n%s" % metrics.confusion_matrix(test_labels_mnist, y_pred_mnist))


Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       164
           1       0.71      0.97      0.82       152
           2       0.84      0.73      0.78       155
           3       0.74      0.76      0.75       154
           4       0.75      0.76      0.75       143
           5       0.72      0.69      0.70       141
           6       0.90      0.86      0.88       143
           7       0.95      0.80      0.87       158
           8       0.79      0.72      0.75       132
           9       0.76      0.80      0.78       158

    accuracy                           0.80      1500
   macro avg       0.81      0.80      0.80      1500
weighted avg       0.81      0.80      0.80      1500


Confusion matrix SKLearn GNB:
[[150   0   2   0   0   6   3   1   2   0]
 [  0 148   0   0   0   2   0   0   2   0]
 [  0  15 113   8   2   3   3   1   8   2]
 [  1   5   8 117   1   7   1   2   8   4]
 [ 

Gaussian Naive Bayesian Classifier (GNB) 

In [218]:
GNB.fit(train_features_mnist, train_labels_mnist)
y_pred_mnist = GNB.predict(test_features_mnist)

print("Classification report MNIST_Light GNB:\n%s\n"
  % (metrics.classification_report(test_labels_mnist, y_pred_mnist)))
print("Confusion matrix MNIST_Light GNB:\n%s" % metrics.confusion_matrix(test_labels_mnist, y_pred_mnist))


Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       164
           1       0.76      0.97      0.85       152
           2       0.83      0.70      0.76       155
           3       0.76      0.74      0.75       154
           4       0.81      0.64      0.71       143
           5       0.93      0.67      0.78       141
           6       0.84      0.94      0.89       143
           7       0.96      0.81      0.88       158
           8       0.74      0.75      0.74       132
           9       0.66      0.88      0.75       158

    accuracy                           0.81      1500
   macro avg       0.82      0.80      0.80      1500
weighted avg       0.82      0.81      0.81      1500


Confusion matrix SKLearn GNB:
[[156   0   1   0   0   1   2   0   3   1]
 [  0 148   0   0   0   1   1   0   2   0]
 [  1   6 109  11   1   1  13   1  12   0]
 [  0   6  15 114   1   0   2   2   9   5]
 [ 